In [60]:
import numpy as np

def pbb_conversion(totals, successes, sim_count = 10000):
    beta_samples = np.array([np.random.beta(successes[i] + 1/2, totals[i] - successes[i] + 1/2, sim_count) for i in range(len(totals))])
    
    max_values = np.argmax(beta_samples, axis=0)
    unique, counts = np.unique(max_values, return_counts=True)
    ocurrences = dict(zip(unique, counts))
    
    result = []
    for i in range(len(totals)):
        result.append(round(ocurrences.get(i, 0) / sim_count, 5))
    
    return result

In [59]:
pbb_conversion([1000,1000], [200,230], 100000)

[0.05412, 0.94588]

In [55]:
pbb_conversion([100,100], [20,22], 100000)

[0.36307, 0.63693]

In [44]:
import numpy as np

def pbb_conversion(totals, successes, sim_count = 10000):
    beta_samples = np.array([np.random.beta(successes[i] + 1/2, totals[i] - successes[i] + 1/2, sim_count) for i in range(len(totals))])
    
    max_values = np.argmax(beta_samples, axis=0)
    unique, counts = np.unique(max_values, return_counts=True)
    ocurrences = dict(zip(unique, counts))
    
    result = []
    for i in range(len(totals)):
        result.append(round(ocurrences.get(i, 0) / sim_count, 5))
    
    return (beta_samples[0] - beta_samples[1])  #/ beta_samples[0]

In [45]:
arr = pbb_conversion([1000,1000], [197,230], 1000000)

In [46]:
#np.mean(arr)
np.mean(arr[arr>0])
np.mean(arr[arr<0])

0.0073162350848118155

-0.034477401964820825

In [64]:
import numpy as np

def pbb_conversion_loss(totals, successes, sim_count = 10000):
    beta_samples = np.array([np.random.beta(successes[i] + 1/2, totals[i] - successes[i] + 1/2, sim_count) for i in range(len(totals))])
    
    max_values = np.argmax(beta_samples, axis=0)
    unique, counts = np.unique(max_values, return_counts=True)
    ocurrences = dict(zip(unique, counts))
    
    result = []
    for i in range(len(totals)):
        result.append(round(ocurrences.get(i, 0) / sim_count, 5))
        
    A = beta_samples[0]
    B = beta_samples[1]
    A_AoverB = (A * (A>B))
    B_AoverB = (B * (A>B))
    A_AoverB_nonzero = A_AoverB[A_AoverB != 0]
    B_AoverB_nonzero = B_AoverB[B_AoverB != 0]
    
    f_mean = np.mean((A_AoverB_nonzero - B_AoverB_nonzero) / B_AoverB_nonzero)
    
    diff_mean = np.mean(A_AoverB_nonzero - B_AoverB_nonzero)
    
    
    
    return diff_mean, f_mean, diff_mean * f_mean, np.mean(A_AoverB - B_AoverB), np.mean(A_AoverB - B_AoverB) / result[0]
#np.mean((A_AoverB_nonzero - B_AoverB_nonzero) * A_AoverB_nonzero * B_AoverB_nonzero)
#(beta_samples[0] - beta_samples[1]) / beta_samples[0]

In [67]:
pbb_conversion_loss([1000,1000], [200,230], 1000000)
pbb_conversion_loss([1000,1000], [197,230], 1000000)

(0.007661636324256129,
 0.036789896451932924,
 0.000281870807021751,
 0.00039131807526138196,
 0.007660886359854776)

(0.0072425821427005614,
 0.03497961289950878,
 0.0002533427197445605,
 0.00026121096755863857,
 0.007241778973070102)

In [33]:
pbb_conversion_loss([1000,1000], [230,200], 1000000)

(0.032027660287368816,
 0.16457240879129686,
 0.0052708692014416456,
 0.030385666199756002,
 0.03375845634413249,
 0.2312932795653056,
 0.19926561927793687)

In [34]:
a = 0.2312932795653056
b = 0.19926561927793687

In [36]:
a/b-1

0.16072848092623726

0.16072848092623723

In [39]:
0.23/b

1.1542382516032264

### pokus podla
https://cdn2.hubspot.net/hubfs/310840/VWO_SmartStats_technical_whitepaper.pdf

In [363]:
totals = [100,100]
successes = [20,22]
#successes = [230,230]

sim_count = 1000

In [364]:
beta_samples = np.array([np.random.beta(successes[i] + 1/2, totals[i] - successes[i] + 1/2, sim_count) for i in range(len(totals))])

posteriorA = beta_samples[0]
posteriorB = beta_samples[1]

In [365]:
joint_posterior = np.zeros(shape=(sim_count, sim_count))

In [366]:
for i in range(sim_count):
    for j in range(sim_count):
        joint_posterior[i,j] = posteriorA[i] * posteriorB[j]

In [367]:
errorFunctionA = 0.0
for i in range(sim_count):
    for j in range(i, sim_count):
        errorFunctionA += joint_posterior[i,j]

In [368]:
def loss(i, j, var):
    if var == 'A':
        return max(j*(1/sim_count) - i*(1/sim_count), 0.0)
    if var == 'B':
        return max(i*(1/sim_count) - j*(1/sim_count), 0.0)

In [369]:
lossFunctionA = 0.0
lossFunctionB = 0.0
for i in range(sim_count):
    for j in range(sim_count):
        lossFunctionA += joint_posterior[i,j] * loss(i,j,'A')
        lossFunctionB += joint_posterior[i,j] * loss(i,j,'B')

In [370]:
lossFunctionA/(sim_count**2)
lossFunctionB/(sim_count**2)

0.007468349302886589

0.007581816459592829

### pokus podla:
https://www.chrisstucchio.com/blog/2014/bayesian_ab_decision_rule.html<br>
https://github.com/Vidogreg/bayes-ab-testing/blob/master/bayes-conversion-test/utilities.R

In [387]:
A_c = 20
A_t = 100
B_c = 22
B_t = 100

alphaA = A_c
alphaB = B_c
betaA = A_t - A_c
betaB = B_t - B_c

pbbs = pbb_conversion([A_t,B_t], [A_c,B_c], 100000)
pbbs

A_better = pbbs[0]
B_better = pbbs[1]

[0.36398, 0.63602]

In [393]:
import math
 
def beta(a,b):
     
    '''uses gamma function or inbuilt math.gamma() to compute values of beta function'''
     
    beta = math.gamma(a)*math.gamma(b)/math.gamma(a+b)
    return beta

In [396]:
(beta(alphaB + 1, betaB) / beta(alphaB, betaB)) * B_better - (beta(alphaA + 1, betaA) / beta(alphaA, betaA)) * A_better

0.0671284